install dependencys

In [ ]:
!pip3 install openai --user
!pip install openpyxl
!pip install requests
!pip install pandas-gbq --user

Inputs

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID


'datalake-analytics-339922'

Input data by Big Query

packages

In [3]:
import openai
import requests
import os
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
print("packages is: ok")

packages is: ok


clients

In [4]:
bq = bigquery.Client(project = PROJECT_ID)
print("client is: ok")

client is: ok


Data on bigquery

Input Data by file xlxs

In [5]:
df_bq_2 = pd.read_excel('data/data_open_ia.xlsx')
df_bq_2.shape


(2541, 27)

clean data

In [7]:
dfcln = df_bq_2[['Id_Model','Message']]
print("dataframe filtred is :ok")

dataframe filtred is :ok


variables open ia:

-ia1 : Message
-ia2 : Message_x
-ia3 : Message_x_x
-ia4 : Message_x_x_x
-ia5 : Message_x_x_x_x
-ia6 : Message_x_x_x_x_x

data_clean

In [8]:
first_batch = dfcln.iloc[0:1000]
first_batch.astype(str)
bct_inx = first_batch.reset_index()
print(bct_inx.head())
#lista
posts = first_batch['Message'].tolist()
print('posts is : ok')
len(posts)

   index                             Id_Model            Message_x_x_x_x_x
0      0  model23020913071100000001879escv5f"          Team Bad Bunny !¡!!
1      1  model23020913071100000001880escv5f"  Trepadora Social y clasista
2      2  model23020913071100000001881escv5f"                 GIFS o Fotos
3      3  model23020913071100000001882escv5f"                         💖💖💖💖
4      4  model23020913071100000001883escv5f"                 GIFS o Fotos
posts is : ok


1000

Make a batch OPEN IA_1

In [ ]:


# Initialize OpenAI API with API key
openai.api_key = "sk-##############################puCw22"

# Define the model engine and the prompt to use
model_engine = "text-davinci-002"
prompt = "Say what is the feeling expressed in the following headline (Positive,Negative,Neutral): "

# Create a list of posts/titles to classify
posts = [] # Add titles to this list

# Initialize a list to store responses
responses = []

# Make a request for each title in the list and store the response
for title in posts:
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt + str(title), # Concatenate the prompt and the title
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.7,
    )
    # Append the response to the list
    responses.append(response.choices[0].text)

# Print a message to indicate that data processing is complete
print("Data processing is complete")

# Clean up the response
mi_lista_1 = [re.sub('\n\n', '', x) for x in responses]

# Print a message to indicate that the first list has been created
print("The first list has been created")

# Print the length of the first list
print(len(mi_lista_1))

# Create a dataframe from the first list
df_text_davinci_002 = pd.DataFrame(mi_lista_1, columns=['Clasify_IA_1'])

# Print the length of the dataframe
print(len(df_text_davinci_002))

# Create an empty dataframe to store the classification results
df_num = pd.DataFrame()

# Fill the dataframe with zeros
df_num = df_num.fillna(0)

# Extract the classification results from the dataframe and store them in a new dataframe
for i in df_text_davinci_002.columns:
    df_num[i] = df_text_davinci_002 [i].str.extract(r'(Rabia|Disgusto|Miedo|Alegría|Tristeza|Ninguno)')

# Fill any missing values with "Ninguno"
df_num_v1 = df_num.fillna("Ninguno")

# Concatenate the new dataframe with another dataframe (bct_inx)
last_df = pd.concat([df_num_v1, bct_inx], axis=1)

# Print the length of the final dataframe
print(len(last_df))

# Print a message to indicate that the classification is complete
print('Classification is complete')

# Merge two dataframes (df_bq_2 and last_df) on the "Id_Model" column
result = pd.merge(left=df_bq_2, right=last_df, left_on='Id_Model', right_on='Id_Model', how='right')

#delete excedent columns and print the shape of dataframe
df_result_v1 = result.drop(["Message_y","index"], axis=1)
# Print the shape of the final dataframe
print("The final dataframe has shape:", df_result_v1.shape)

Make a batch OPEN IA_2

In [33]:
# Define your OpenAI API key
openai.api_key = "sk-##############################puCw22"

# Define the model and prompt you want to use
model_engine = "text-davinci-002"
prompt = "Diga cual es el sentimiento expresado en el siguiente titular (Positivo,Negativo,Neutro):"

# Create a list of headlines for your "batch"
posts = []

# Create an empty list to store the responses
responses = []

# Make a request for each title in the list
for title in posts:
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt + str(title),
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.7,
    )
    
    # Add the response to the list
    responses.append(response.choices[0].text)

print("Data retrieval completed successfully.")

# Use a regex and the "re.sub()" function to replace all occurrences of "\n\n"
mi_lista_1 = [re.sub('\n\n', '', x) for x in responses]

print("First list is complete.")

# Join to dataframe
# We join this df by index
df_text_davinci_002 = pd.DataFrame(mi_lista_1, columns=['Clasify_IA_2'])

print("Dataframe created successfully.")

df_num = pd.DataFrame()
df_num = df_num.fillna(0)

for i in df_text_davinci_002.columns:
    df_num[i] = df_text_davinci_002 [i].str.extract(r'(Positivo|Negativo|Neutro)')

df_num_v1 = df_num.fillna("Ninguno")
last_df = pd.concat([df_num_v1, bct_inx], axis=1)

print("Data processing complete.")

result = pd.merge(left=df_bq_2, right=last_df, left_on='Id_Model', right_on='Id_Model', how='right')

print("Data merging complete.")

df_result_v1 = result.drop(["Message_x_y","index"], axis=1)
print("Data cleaning complete.")

# Print the shape of the final dataframe
print("The final dataframe has shape:", df_result_v1.shape)

data is :ok


make a batch ia3

In [57]:


# Inicializar la API de OpenAI
openai.api_key = "sk-B###########################5gg7puCw22"

# Definir el modelo y el prompt que quieres utilizar
model_engine = "text-davinci-002"
prompt = "depen how do you like to clasify"


# Crear una lista de titulares para tu "batch"



# Crear una lista vacía para almacenar las respuestas
responses = []

# Realizar una petición para cada título en la lista
for title in posts:
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt +str(title),
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.7,
        
    )
    
    responses.append(response.choices[0].text)
print("data is :ok")

#clean response
# Podemos usar una regex y la función "re.sub()" para reemplazar todas las ocurrencias de "\n\n"
mi_lista_1 = [re.sub('\n\n|\n|\n2|\n3|1.', '', x) for x in responses]
print("primera lista is :ok")
print(len(mi_lista_1))

#joint to dataframe
#we joint this df by index
df_text_davinci_002 = pd.DataFrame(mi_lista_1, columns=['Clasify_IA_3'])
print(len(df_text_davinci_002))
df_num_v1 = df_text_davinci_002.fillna("Ninguno")
last_df_1 = pd.concat([df_num_v1, bct_inx], axis=1)
#print(last_df_1.shape)
result = pd.merge(left=df_bq_2, right=last_df_1, left_on='Id_Model', right_on='Id_Model', how='right')
df_result_v1 = result.drop(["Message_x_x_y","index"], axis=1)
df_result_v1.shape

data is :ok
primera lista is :ok
541
541


(541, 25)

make a batch ia4

In [84]:


# Inicializar la API de OpenAI
openai.api_key = "sk-Bt#########################puCw22"

# Definir el modelo y el prompt que quieres utilizar
model_engine = "text-davinci-002"
prompt = "depen how do you like to clasify"


# Crear una lista de titulares para tu "batch"



# Crear una lista vacía para almacenar las respuestas
responses = []

# Realizar una petición para cada título en la lista
for title in posts:
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt +str(title),
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.7,
        
    )
    
    responses.append(response.choices[0].text)
print("data is :ok")

#clean response
# Podemos usar una regex y la función "re.sub()" para reemplazar todas las ocurrencias de "\n\n"
mi_lista_1 = [re.sub('\n\n|\n|\n2|\n3|1.', '', x) for x in responses]
print("primera lista is :ok")
len(mi_lista_1)

#joint to dataframe
#we joint this df by index
df_text_davinci_002 = pd.DataFrame(mi_lista_1, columns=['Clasify_IA_4'])
print(len(df_text_davinci_002))
df_num = pd.DataFrame()
df_num = df_num.fillna(0)
for i in df_text_davinci_002.columns:
    df_num[i] = df_text_davinci_002 [i].str.extract(r'(Ciencia|Cobertura|Cultura|Economía|Educación|Finanzas|Ley y Justicia|Medio Ambiente|Negocios|Política|Promociones|Quejas|Salud|Satisfacción del cliente|Tecnología|Turismo|Vivienda)')
#df_text_davinci_002

df_num_v1 = df_num.fillna("Ninguno")
last_df = pd.concat([df_num_v1, bct_inx], axis=1)
print(len(last_df))
print('clasify is okey')
last_df.tail()
result = pd.merge(left=df_bq_2, right=last_df, left_on='Id_Model', right_on='Id_Model', how='right')

#result
df_result_v1 = result.drop(["Message_x_x_x_y","index"], axis=1)
df_result_v1.shape
#df_result = result.drop(result.index[0])


data is :ok
primera lista is :ok
541
541
clasify is okey


(541, 26)

make a batch ia5

In [100]:


# Inicializar la API de OpenAI
openai.api_key = "sk-BtBV###################3g7puCw22"

# Definir el modelo y el prompt que quieres utilizar
model_engine = "text-davinci-002"
prompt = "depen how do you like to clasify"


# Crear una lista de titulares para tu "batch"



# Crear una lista vacía para almacenar las respuestas
responses = []

# Realizar una petición para cada título en la lista
for title in posts:
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt +str(title),
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.7,
        
    )
    
    responses.append(response.choices[0].text)
print("data is :ok")

#clean response
# Podemos usar una regex y la función "re.sub()" para reemplazar todas las ocurrencias de "\n\n"
mi_lista_1 = [re.sub('\n\n|\n|\n2|\n3|1.', '', x) for x in responses]
print("primera lista is :ok")
print(len(mi_lista_1))
#joint to dataframe
#we joint this df by index
df_text_davinci_002 = pd.DataFrame(mi_lista_1, columns=['Clasify_IA_5'])
print(len(df_text_davinci_002))
df_num = pd.DataFrame()
df_num = df_num.fillna(0)
for i in df_text_davinci_002.columns:
    df_num[i] = df_text_davinci_002 [i].str.extract(r'(Sorpresa|Alegría|Satisfacción|Admiración|Curiosidad|Tristeza|Aversión|Enojo|Miedo|Amor|Ansiedad|Ninguno)')
#df_text_davinci_002#

df_num_v1 = df_num.fillna("Ninguno")
last_df = pd.concat([df_num_v1, bct_inx], axis=1)
print(len(last_df))
print('clasify is okey')
last_df.tail()
result = pd.merge(left=df_bq_2, right=last_df, left_on='Id_Model', right_on='Id_Model', how='right')

#result
df_result_v1 = result.drop(["Message_x_x_x_x_y","index"], axis=1)
df_result_v1.shape
#df_result = result.drop(result.index[0])

data is :ok
primera lista is :ok
541
541
541
clasify is okey


(541, 27)

make a batch ia6

In [ ]:

print ('the process started')
# Inicializar la API de OpenAI
openai.api_key = "sk-B########################7puCw22"

# Definir el modelo y el prompt que quieres utilizar
model_engine = "text-davinci-002"
prompt = "depen how do you like to clasify"

#Defina una categoría que permita saber el siguiente texto a 
# (): "TEXT HERE"
# Crear una lista de titulares para tu "batch"



# Crear una lista vacía para almacenar las respuestas
responses = []

# Realizar una petición para cada título en la lista
for title in posts:
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt +str(title),
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.7,
        
    )
    
    responses.append(response.choices[0].text)
    
print("data is :ok")


the process started


In [ ]:


#clean response
# Podemos usar una regex y la función "re.sub()" para reemplazar todas las ocurrencias de "\n\n"
mi_lista_1 = [re.sub('\n\n|\n|\n2|\n3|1.', '', x) for x in responses]
print("primera lista is :ok")
print(len(mi_lista_1))
#joint to dataframe
#we joint this df by index
df_text_davinci_002 = pd.DataFrame(mi_lista_1, columns=['Clasify_IA_6'])
print(len(df_text_davinci_002))
df_num = pd.DataFrame()
df_num = df_num.fillna(0)
for i in df_text_davinci_002.columns:
    df_num[i] = df_text_davinci_002 [i].str.extract(r'(Adultos|Arte y Entretenimiento|Coches y Vehículos|Belleza y Fitness|Libros y literatura|Negocios e Industria|Informática y electrónica|Finanzas|Alimentación|Juegos|Salud|Ocio y tiempo libre|Hogar y jardín|Internet y Telecomunicaciones|Empleo y Educación|Derecho y Gobierno|Noticias|Comunidades en línea|Gente y sociedad|Animales y mascotas|Inmobiliaria|Referencia|Ciencia|Temas delicados|Compras|Deportes|Viajes y transportes)')
#df_text_davinci_002#

df_num_v1 = df_num.fillna("Ninguno")
last_df = pd.concat([df_num_v1, bct_inx], axis=1)
print(len(last_df))
print('clasify is okey')
last_df.tail()
result = pd.merge(left=df_bq_2, right=last_df, left_on='Id_Model', right_on='Id_Model', how='right')
#print(result)
#result

#df_result = result.drop(result.index[0])

In [56]:
df_result_v1 = result.drop(["Message_x_x_x_x_x_y","index"], axis=1)
df_result_v1.shape

(53, 93)

Assing the DataFrame processed to a new variable to contact with the df_final we just neeet to execute the first time

In [92]:
df_final = pd.DataFrame()
print('df_final is ok')

df_final is ok


then to execute the first time the variable we change df_final by merged_df

In [101]:
df1 = df_result_v1
merged_df = pd.concat([df1,merged_df], axis=0)
merged_df.shape

(2541, 27)

In [102]:
merged_df.head(5)

,Id_Model,User,Message_x_x_x_x_x,Brands,Subsidiary,Sentimiento,Topics 1,Topics 2,Topics 3,Locations,...,Post Id 2,First Admin Comment,Link,Sentiment,Clasification_M,Clasify_IA_1,Clasify_IA_2,Clasify_IA_3,Clasify_IA_4,Clasify_IA_5
0,"model23020913071100000001879escv5f""",Facebook User,Team Bad Bunny !¡!!,Point Guard,PROPIEDAD,Neutral,Comunicación oficial,Tendencias,NaN,NaN,...,NaN,NaN,https://www.facebook.com/556175296544659#chann...,1,Menciones generales,Alegría,Positivo,Bad Bunny2. Team3. !¡!!,Cultura,Ninguno
1,"model23020913071100000001880escv5f""",Facebook User,Trepadora Social y clasista,Point Guard,PROPIEDAD,Neutral,Comunicación oficial,Tendencias,NaN,NaN,...,NaN,NaN,https://www.facebook.com/556175296544659#chann...,0,Institucional,Disgusto,Negativo,"social, clasista, trepadora",Política,Ninguno
2,"model23020913071100000001881escv5f""",Facebook User,GIFS o Fotos,Point Guard,PROPIEDAD,Neutral,Comunicación oficial,Tendencias,NaN,NaN,...,NaN,NaN,https://www.facebook.com/556175296544659#chann...,2,Institucional,Ninguno,Neutro,GIFS2. Fotos3.,Cultura,Sorpresa
3,"model23020913071100000001882escv5f""",Facebook User,💖💖💖💖,Point Guard,PROPIEDAD,Neutral,Comunicación oficial,Tendencias,NaN,NaN,...,NaN,NaN,https://www.facebook.com/556175296544659#chann...,2,Comunicación oficial,Alegría,Positivo,Bridget Jones's Diary is a 96 novel by Helen F...,Cultura,Amor
4,"model23020913071100000001883escv5f""",Facebook User,GIFS o Fotos,Point Guard,PROPIEDAD,Neutral,Comunicación oficial,Tendencias,NaN,NaN,...,NaN,NaN,https://www.facebook.com/556175296544659#chann...,2,Institucional,Ninguno,Positivo,"Fotos, GIFS, o",Ninguno,Alegría


Export dataframe to a file xlxs is a better option to create a new table o Big Query just make this part whe all data is jointed

In [104]:
merged_df.to_excel('data/data_open_ia.xlsx',index=False)
print('data is on file : ok')

data is on file : ok
